In [1]:
import requests
import pymorphy2

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
MODEL = 'ruscorpora_upos_cbow_300_20_2019'
FORMAT = 'csv'

def api_neighbor(m, w, f):
    neighbors = []
    url = '/'.join(['http://rusvectores.org', m, w, 'api', f]) + '/'
    r = requests.get(url=url, stream=True)
    
    if len(r.text.split('\n')) == 1:
        return 'none'
    
    orig_word = r.text.split('\n')[1]
    pos = orig_word.split('_')[1]
    
    for line in r.text.split('\n')[2:-1]:
        word, sim = line.split('\t')
        if pos in word:
            neighbors.append(word)
    if 'VERB' == pos:
        return neighbors[1]
    return neighbors[0]

def find_synonyms(s):
    modified_s = []

    for word in s.split(' '):

        w = morph.parse(word)[0]
        normal_form = w.normal_form

        modified_word = w.word
        if w.tag.POS in ['INFN', 'VERB']:
            sim = api_neighbor(MODEL, normal_form, FORMAT)
            if sim != 'none':
                modified_word = morph.parse(sim.split('_')[0])[0].inflect({w.tag.tense, w.tag.mood}).word
        elif w.tag.POS == 'NOUN':
            if 'Name' in w.tag or 'Geox' in w.tag or 'Orgn' in w.tag:
                pass
            else:
                sim = api_neighbor(MODEL, normal_form, FORMAT)
                if sim != 'none':
                    modified_word = morph.parse(sim.split('_')[0])[0].inflect({w.tag.case,w.tag.number}).word
        elif w.tag.POS == 'ADJF':
            sim = api_neighbor(MODEL, normal_form, FORMAT)
            if sim != 'none':
                if morph.parse(sim.split('_')[0])[0].inflect({w.tag.case,w.tag.number}):
                    modified_word = morph.parse(sim.split('_')[0])[0].inflect({w.tag.case,w.tag.number}).word
        modified_s.append(modified_word)
    
    return ' '.join(modified_s)

In [4]:
phrases = [
    'В Москве в воскресенье днем начнется сильный снегопад его принесет циклон Сара сообщил РИА Новости ведущий сотрудник центра погоды Фобос Евгений Тишковец .',
    'Собянин отметил , что принятые ограничительные меры крайне неприятны, однако необходимы . Он уточнил , что за последние две недели прирост новых заболевших составил около 6% . По мнению мэра , выявляемость заболевших COVID-19 в Москве находится на одном из лучших уровней по стране .',
    'Музыкальный продюсер во время интервью заявил , что участия Киркорова , Меладзе , Баскова и прочих аналогичных звезд в праздничных программах совсем не обязательно . Из года в год на всех каналах одни и те же лица , причем круглогодично .'
]

In [5]:
for phrase in phrases:
    print(phrase)
    print()
    print(find_synonyms(phrase))
    print('==========')

В Москве в воскресенье днем начнется сильный снегопад его принесет циклон Сара сообщил РИА Новости ведущий сотрудник центра погоды Фобос Евгений Тишковец .

в москве в воскресение неделей возобновлюсь слабый метель его принешу антициклон сара извещали риа известия телепрограмма руководитель центра ненастья фобос евгений тишковец .
Собянин отметил , что принятые ограничительные меры крайне неприятны, однако необходимы . Он уточнил , что за последние две недели прирост новых заболевших составил около 6% . По мнению мэра , выявляемость заболевших COVID-19 в Москве находится на одном из лучших уровней по стране .

собянин отмечались , что принятые запретительные меры крайне неприятны, однако необходимы . он поинтересовались , что за последующих две месяца рост обновляющих заболевших составлялись около 6% . по суждению мэрии , выявляемость заболевших covid-19 в москве имеюсь на одном из плохих показателей по государстве .
Музыкальный продюсер во время интервью заявил , что участия Киркорова

## Результат

Находил синонимы с помощью модели ruscorpora_upos_cbow_300_20_2019.
В коде выше старался исключать из нахождения синонимов имена собственные и ставить существительные и глаголы в нужные формы, но получилось как-то не очень.
В итоге, выданный текст немного поправил.

Заметил, что данная модель на прилагательных первым схожим словом выдает прилагательное антоним (их не стал оставлять, так как от этого меняется смысл нововсти). 

Также так как модель не учитывает контекст предложения, в котором стоит слово иногда некорректно выдает синоним, например, для слова в фразе 3 "канал" модель выдала "шлюз", хотя в предложении шла речь о телевизионном канале. Но в некоторых местах получилось забавно: центр погоды - центр ненастья (фраза 1), одни и те же лица - одни и те же физиономии (фраза 3)

#### Фраза 1

**Оригинал:**

В Москве в воскресенье днем начнется сильный снегопад. Его принесет циклон Сара сообщил РИА Новости ведущий сотрудник центра погоды Фобос Евгений Тишковец.

**Фраза с синонимами:**

В Москве в воскресенье днем возобновится сильная метель. Ее принесет антициклон Сара извещает РИА Новсти ведущий руководитель центра ненастья фобос Евгений Тишковец.

#### Фраза 2

**Оригинал:**

Собянин отметил, что принятые ограничительные меры крайне неприятны, однако необходимы. Он уточнил, что за последние две недели прирост новых заболевших составил около 6%. По мнению мэра, выявляемость заболевших COVID-19 в Москве находится на одном из лучших уровней по стране.

**Фраза с синонимами:**

Собянин отмечал, что принятые запретительные меры крайне неприятны, однако необходимы. Он поинтересовался, что за последние две недели рост обновленных заболевших составлял около 6%. По суждению мэрии, выявляемость заболевших covid-19 в Москве имеется на одном из лучших показателей по государству.

#### Фраза 3

**Оригинал:**

Музыкальный продюсер во время интервью заявил, что участия Киркорова, Меладзе, Баскова и прочих аналогичных звезд в праздничных программах совсем не обязательно. Из года в год на всех каналах одни и те же лица, причем круглогодично.

**Фраза с синонимами:**

Фортепианная кинокомпания во время интервью объявила, что участие Киркорова, Меладзе, Баскова и коих сходных небосводов в предпраздничных проектах совсем не обязательно. Из года в год на всех каналах одни и те же физиономии, причём круглогодично.